<a href="https://colab.research.google.com/github/11doris/jazz-maestro/blob/colab_word_embeddings/section_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sections as Input

In [3]:
!pip uninstall gensim -y

Found existing installation: gensim 4.1.2
Uninstalling gensim-4.1.2:
  Successfully uninstalled gensim-4.1.2


In [4]:
!pip install gensim

  Using cached gensim-4.1.2-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (24.1 MB)


In [2]:
import gensim
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pprint
import pandas as pd
import numpy as np
from collections import Counter
import plotly.express as px
from tqdm import tqdm 
from gensim.models.doc2vec import Doc2Vec
import pickle
import os

In [3]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [4]:
print(gensim.__version__)

4.1.2


# Initialization

## Download the Data

### Download Data

### Simplified Chords

In [14]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1lVm6r0T3i1hbyZqPJXNKf90uh14MOKkb' -O data.csv

--2021-11-19 12:01:25--  https://docs.google.com/uc?export=download&id=1lVm6r0T3i1hbyZqPJXNKf90uh14MOKkb
Resolving docs.google.com (docs.google.com)... 172.217.214.139, 172.217.214.138, 172.217.214.113, ...
Connecting to docs.google.com (docs.google.com)|172.217.214.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0c-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/f548h6gnpb05b7bth99kbk8efgjar421/1637323275000/14329102864480165501/*/1lVm6r0T3i1hbyZqPJXNKf90uh14MOKkb?e=download [following]
--2021-11-19 12:01:26--  https://doc-0c-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/f548h6gnpb05b7bth99kbk8efgjar421/1637323275000/14329102864480165501/*/1lVm6r0T3i1hbyZqPJXNKf90uh14MOKkb?e=download
Resolving doc-0c-4c-docs.googleusercontent.com (doc-0c-4c-docs.googleusercontent.com)... 142.250.136.132, 2607:f8b0:4001:c34::84
Connecting to doc-0c-4c-docs.googleusercontent.com (doc-0c

### Basic Chords (major triads, minor triads, dom7)

In [16]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1lQTBMYk7Ui_mbOopUTjOs4MV_4c1a8bO' -O data.csv

--2021-11-19 12:01:30--  https://docs.google.com/uc?export=download&id=1lQTBMYk7Ui_mbOopUTjOs4MV_4c1a8bO
Resolving docs.google.com (docs.google.com)... 172.217.214.139, 172.217.214.102, 172.217.214.100, ...
Connecting to docs.google.com (docs.google.com)|172.217.214.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0g-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/0vt104mnrduin73h6ut9g947m619vput/1637323275000/14329102864480165501/*/1lQTBMYk7Ui_mbOopUTjOs4MV_4c1a8bO?e=download [following]
--2021-11-19 12:01:31--  https://doc-0g-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/0vt104mnrduin73h6ut9g947m619vput/1637323275000/14329102864480165501/*/1lQTBMYk7Ui_mbOopUTjOs4MV_4c1a8bO?e=download
Resolving doc-0g-4c-docs.googleusercontent.com (doc-0g-4c-docs.googleusercontent.com)... 142.250.136.132, 2607:f8b0:4001:c34::84
Connecting to doc-0g-4c-docs.googleusercontent.com (doc-0g

### Basic Plus Chords
M7 and 6 reduced to major triad, m7 reduced to m, dominant 7, m7b5, diminished, and all (b5) left as they are.

In [63]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=17djlZRWFSUCviOxRTOF-nwbbZqU5gUP9' -O data.csv

--2021-11-19 12:06:23--  https://docs.google.com/uc?export=download&id=17djlZRWFSUCviOxRTOF-nwbbZqU5gUP9
Resolving docs.google.com (docs.google.com)... 172.217.214.102, 172.217.214.100, 172.217.214.139, ...
Connecting to docs.google.com (docs.google.com)|172.217.214.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/npg59itir7fld5of4s6ivpcd3tqiabdo/1637323575000/14329102864480165501/*/17djlZRWFSUCviOxRTOF-nwbbZqU5gUP9?e=download [following]
--2021-11-19 12:06:23--  https://doc-0s-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/npg59itir7fld5of4s6ivpcd3tqiabdo/1637323575000/14329102864480165501/*/17djlZRWFSUCviOxRTOF-nwbbZqU5gUP9?e=download
Resolving doc-0s-4c-docs.googleusercontent.com (doc-0s-4c-docs.googleusercontent.com)... 142.250.136.132, 2607:f8b0:4001:c34::84
Connecting to doc-0s-4c-docs.googleusercontent.com (doc-0s

### Read Chords Input Data

In [64]:
path_to_file = '/content/data.csv'
df = pd.read_csv(path_to_file, sep='\t', index_col="id")
df = df.reset_index()

In [65]:
df

,id,file_name,title,tune_mode,tune_id,section_name,section_id,chords
0,0,dataset/jazz1350/26-2.xml,26-2,major,0,A,1,C Eb7 G# B7 E G7 Gm C7 F G#7 C# E7 Am D7 Dm G7
1,1,dataset/jazz1350/26-2.xml,26-2,major,0,A,2,C Eb7 G# B7 E G7 Gm C7 F Eb7 G# B7 E G7 C
2,2,dataset/jazz1350/26-2.xml,26-2,major,0,B,3,Gm C7 Bm E7 A C7 F Bbm Eb7 G# Dm G7
3,3,dataset/jazz1350/26-2.xml,26-2,major,0,A,4,C Eb7 G# B7 E G7 Gm C7 F Eb7 G# B7 E G7 C
4,4,dataset/jazz1350/500 Miles High.xml,500 Miles High,minor,1,NaN,0,Am Am Cm Cm Eb Eb Em7b5 A7 Dm Dm Bm7b5 Bm7b5 B...
...,...,...,...,...,...,...,...,...
5398,5398,dataset/trad/You're Lucky To Me.xml,You're Lucky To Me,major,1808,A,1,Dm G7 Em7b5 A7 Dm G#7 G7 C C
5399,5399,dataset/trad/You're Lucky To Me.xml,You're Lucky To Me,major,1808,A,2,Dm G7 Em7b5 A7 Dm G#7 G7 C C A7 A7 A7 A7 Dm G7...
5400,5400,dataset/trad/You're Nobody Till Somebody Loves...,You're Nobody Till Somebody Loves You,major,1809,A,1,C E7 A7 A7 Dm G7 C Em C G7 Em Cdim Dm Dm Dm D7...
5401,5401,dataset/trad/Your Feet's Too Big.xml,Your Feet's Too Big,major,1810,NaN,0,G7 G7 C C G7 G7 C F G# C C G7 G7 C F C C G7 C ...


### Meta Data

In [66]:
titles = df.loc[:, ['id', 'tune_id', 'section_id', 'section_name', 'title', 'tune_mode']]

In [67]:
titles

,id,tune_id,section_id,section_name,title,tune_mode
0,0,0,1,A,26-2,major
1,1,0,2,A,26-2,major
2,2,0,3,B,26-2,major
3,3,0,4,A,26-2,major
4,4,1,0,NaN,500 Miles High,minor
...,...,...,...,...,...,...
5398,5398,1808,1,A,You're Lucky To Me,major
5399,5399,1808,2,A,You're Lucky To Me,major
5400,5400,1809,1,A,You're Nobody Till Somebody Loves You,major
5401,5401,1810,0,NaN,Your Feet's Too Big,major


In [68]:
##TODO Now a tune contains multiple rows

In [69]:
titles_dict = titles.to_dict()
sectionid_to_title = titles_dict['title']
sectionid_to_title[1170]

'Emily'

In [70]:
sectionid_to_title_id = titles_dict['tune_id']
sectionid_to_title_id[5396]

1807

In [71]:
titles_rows = titles.to_dict(orient='records')
titles_rows[0]

sectionid_to_section = []
for i, row in enumerate(titles_rows):
  name = f"{row['title']}, section{row['section_id']} ({row['section_name']})"
  sectionid_to_section.append(name)

sectionid_to_section[3]

'26-2, section4 (A)'

In [72]:
title_to_sectionid = {}

for row in titles.iterrows():
  title = row[1]['title']
  if title not in title_to_sectionid:
    title_to_sectionid[title] = [row[1]['id']]
  else:
    title_to_sectionid[title].append(row[1]['id'])


# List Tunes for Test Data

In [73]:
contrafacts = {
               "26-2": "Confirmation",
               "52nd Street Theme": "I Got Rhythm",
               "Ablution": "All The Things You Are",
               "Anthropology": "I Got Rhythm",
               "Ballade": "As Long As I Live",
               "Bright Mississippi": "Sweet Georgia Brown",
               "C.T.A.": "I Got Rhythm",
               "Celia": "I Got Rhythm",
               "Cottontail": "I Got Rhythm",
               "Countdown": "Tune Up",
               "Dewey Square": "Oh, Lady Be Good",
               "Dexterity": "I Got Rhythm",
               "Dig": "Sweet Georgia Brown",
               "Donna Lee": "Indiana (Back Home Again In)",
               "Don't Be That Way": "I Got Rhythm",
               "Eternal Triangle": "I Got Rhythm",
               "Evidence": "Just You, Just Me",
               "Flintstones": "I Got Rhythm",
               "Four On Six": "Summertime",
               "Freight Train": "Blues For Alice",
               "Good Bait": "I Got Rhythm",
               "Hackensack": "Oh, Lady Be Good",
               "Half Nelson": "Lady Bird",
               "Hot House": "What Is This Thing Called Love",
               "Impressions": "So What",
               "In A Mellow Tone (In A Mellotone)": "Rose Room",
               "In Walked Bud": "Blue Skies",
               "Ko Ko": "Cherokee",
               "Lennie's Pennies": "Pennies From Heaven",   ## Lennie's Pennies is in minor and therefore transposed to Amin... not possible to recognize like that
               # "Let's Call This": "Honeysuckle Rose",
               "Little Rootie Tootie": "I Got Rhythm",
               "Little Willie Leaps": "All God's Chillun Got Rhythm",
               "Lullaby Of Birdland": "Love Me Or Leave Me",
               "Moose The Mooche": "I Got Rhythm",
               "My Little Suede Shoes": "Jeepers Creepers",
               "Oleo": "I Got Rhythm",
               "Ornithology": "How High The Moon",
               "Passport": "I Got Rhythm",
               "Quasimodo (Theme)": "Embraceable You",
               "Rhythm-a-ning": "I Got Rhythm",
               "Room 608": "I Got Rhythm",
               "Salt Peanuts": "I Got Rhythm",
               "Satellite": "How High The Moon",
               "Scrapple From The Apple": "Honeysuckle Rose", # A section
               "Scrapple From The Apple": "I Got Rhythm", # B section
               #"Segment": "I Got Rhythm",
               "Seven Come Eleven": "I Got Rhythm",
               "Shaw 'Nuff": "I Got Rhythm",
               "Theme, The": "I Got Rhythm",
               "Tour De Force": "Jeepers Creepers",
               "Wow": "You Can Depend On Me",
               "Yardbird Suite": "Rosetta",
               # following tunes are not from wikipedia:
               "Sweet Sue, Just You": "Honeysuckle Rose",  # A section
               # "All Of Me": "Pennies From Heaven", # bars 25-28 of All of Me are same as bars 17-20 of Pennies From Heaven, but different key!
               "Sweet Sue, Just You": "Bye Bye Blackbird", # Bridge same
               "These Foolish Things": "Blue Moon", # first 8 bars same
               
}

contrafacts_test = []
for reference, compare in contrafacts.items():
  contrafacts_test.append([title_to_sectionid[reference], title_to_sectionid[compare]])
  

In [74]:
contrafacts_test[:5]

[[[0, 1, 2, 3], [810, 811, 812, 813]],
 [[6, 7, 8, 9], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]],
 [[104, 105, 106, 107], [209, 210, 211, 212]],
 [[272, 273, 274, 275], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]],
 [[335, 336], [297, 298, 299, 300, 4475, 4476, 4477]]]

## Helpers for N-Grams

In [75]:
def ngrams(tokens, n=2, sep='-'):
    return [sep.join(ngram) for ngram in zip(*[tokens[i:] for i in range(n)])]

In [76]:
def list_to_ngrams(data, n=2, sep='-'):
  out = []
  for row in data:
    out.append(ngrams(row, n, sep))

  return out

# TF-IDF



## Variant 1: Build Model using gensim Tfidf

In [502]:
lines = df.loc[:, 'chords'].tolist()
data = [line.split(' ') for line in lines]

In [550]:
ngrams_for_input = [2, 3]

processed_corpus = []
for line in data:
  tune_n = []
  for n in ngrams_for_input:
    tune_n.extend(ngrams(line, n=n))
  processed_corpus.append(tune_n)

for line in processed_corpus[:5]:
  print(line)

['C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm', 'Gm-C7', 'C7-F', 'F-G#7', 'G#7-C#', 'C#-E7', 'E7-Am', 'Am-D7', 'D7-Dm', 'Dm-G7', 'C-Eb7-G#', 'Eb7-G#-B7', 'G#-B7-E', 'B7-E-G7', 'E-G7-Gm', 'G7-Gm-C7', 'Gm-C7-F', 'C7-F-G#7', 'F-G#7-C#', 'G#7-C#-E7', 'C#-E7-Am', 'E7-Am-D7', 'Am-D7-Dm', 'D7-Dm-G7']
['C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm', 'Gm-C7', 'C7-F', 'F-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-C', 'C-Eb7-G#', 'Eb7-G#-B7', 'G#-B7-E', 'B7-E-G7', 'E-G7-Gm', 'G7-Gm-C7', 'Gm-C7-F', 'C7-F-Eb7', 'F-Eb7-G#', 'Eb7-G#-B7', 'G#-B7-E', 'B7-E-G7', 'E-G7-C']
['Gm-C7', 'C7-Bm', 'Bm-E7', 'E7-A', 'A-C7', 'C7-F', 'F-Bbm', 'Bbm-Eb7', 'Eb7-G#', 'G#-Dm', 'Dm-G7', 'Gm-C7-Bm', 'C7-Bm-E7', 'Bm-E7-A', 'E7-A-C7', 'A-C7-F', 'C7-F-Bbm', 'F-Bbm-Eb7', 'Bbm-Eb7-G#', 'Eb7-G#-Dm', 'G#-Dm-G7']
['C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm', 'Gm-C7', 'C7-F', 'F-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-C', 'C-Eb7-G#', 'Eb7-G#-B7', 'G#-B7-E', 'B7-E-G7', 'E-G7-Gm', 'G7-Gm-C7', 'Gm-C7-F', 'C

In [551]:
from gensim import corpora

dictionary = corpora.Dictionary(processed_corpus)
print(dictionary)

2021-11-19 15:53:29,889 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-11-19 15:53:30,162 : INFO : built Dictionary(10302 unique tokens: ['Am-D7', 'Am-D7-Dm', 'B7-E', 'B7-E-G7', 'C#-E7']...) from 5403 documents (total 149371 corpus positions)
2021-11-19 15:53:30,164 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(10302 unique tokens: ['Am-D7', 'Am-D7-Dm', 'B7-E', 'B7-E-G7', 'C#-E7']...) from 5403 documents (total 149371 corpus positions)", 'datetime': '2021-11-19T15:53:30.164458', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'created'}


Dictionary(10302 unique tokens: ['Am-D7', 'Am-D7-Dm', 'B7-E', 'B7-E-G7', 'C#-E7']...)


In [552]:
dictionary.token2id

{'Am-D7': 0,
 'Am-D7-Dm': 1,
 'B7-E': 2,
 'B7-E-G7': 3,
 'C#-E7': 4,
 'C#-E7-Am': 5,
 'C-Eb7': 6,
 'C-Eb7-G#': 7,
 'C7-F': 8,
 'C7-F-G#7': 9,
 'D7-Dm': 10,
 'D7-Dm-G7': 11,
 'Dm-G7': 12,
 'E-G7': 13,
 'E-G7-Gm': 14,
 'E7-Am': 15,
 'E7-Am-D7': 16,
 'Eb7-G#': 17,
 'Eb7-G#-B7': 18,
 'F-G#7': 19,
 'F-G#7-C#': 20,
 'G#-B7': 21,
 'G#-B7-E': 22,
 'G#7-C#': 23,
 'G#7-C#-E7': 24,
 'G7-Gm': 25,
 'G7-Gm-C7': 26,
 'Gm-C7': 27,
 'Gm-C7-F': 28,
 'C7-F-Eb7': 29,
 'E-G7-C': 30,
 'F-Eb7': 31,
 'F-Eb7-G#': 32,
 'G7-C': 33,
 'A-C7': 34,
 'A-C7-F': 35,
 'Bbm-Eb7': 36,
 'Bbm-Eb7-G#': 37,
 'Bm-E7': 38,
 'Bm-E7-A': 39,
 'C7-Bm': 40,
 'C7-Bm-E7': 41,
 'C7-F-Bbm': 42,
 'E7-A': 43,
 'E7-A-C7': 44,
 'Eb7-G#-Dm': 45,
 'F-Bbm': 46,
 'F-Bbm-Eb7': 47,
 'G#-Dm': 48,
 'G#-Dm-G7': 49,
 'Gm-C7-Bm': 50,
 'A7-Dm': 51,
 'A7-Dm-Dm': 52,
 'Am-Am': 53,
 'Am-Am-Cm': 54,
 'Am-Cm': 55,
 'Am-Cm-Cm': 56,
 'Bbm-Bbm': 57,
 'Bbm-Bbm-Fm': 58,
 'Bbm-Fm': 59,
 'Bbm-Fm-Fm': 60,
 'Bm7b5-Bbm': 61,
 'Bm7b5-Bbm-Bbm': 62,
 'Bm7b5-Bm7b5': 63,


Vectorize the corpus with a bag-of-words. 
To convert documents to vectors, we’ll use a document representation called bag-of-words. In this representation, each document is represented by one vector: (id, num_occurence).

The function `doc2bow()` simply counts the number of occurrences of each distinct word, converts the word to its integer word id and returns the result as a sparse vector.


Note that this list lives entirely in memory, in most applications you will want a more scalable solution - then use iterators (not done here).

In [553]:
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]
for line in bow_corpus[0:2]:
  print(line)

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1)]
[(2, 2), (3, 2), (6, 1), (7, 1), (8, 1), (13, 2), (14, 1), (17, 2), (18, 2), (21, 2), (22, 2), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1)]


The tf-idf model transforms vectors from the bag-of-words representation to a vector space where the frequency counts are weighted according to the relative rarity of each word in the corpus.

### Trials, not needed for the Similarity later

In [554]:
from gensim import models

# train the model
tfidf = models.TfidfModel(bow_corpus)

# transform a sample chord string
words = "C Dm C#7".split()

print(tfidf[dictionary.doc2bow(words)])

2021-11-19 15:53:31,547 : INFO : collecting document frequencies
2021-11-19 15:53:31,552 : INFO : PROGRESS: processing document #0
2021-11-19 15:53:31,611 : INFO : TfidfModel lifecycle event {'msg': 'calculated IDF weights for 5403 documents and 10302 features (117438 matrix non-zeros)', 'datetime': '2021-11-19T15:53:31.611111', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'initialize'}


[]


The first entry is the token ID and the second entry is the tf-idf weighting.

C occurs more frequently than Dm, and Dm occurs more frequently than C#7, therefore it has been assigned a lower weight. 

In [555]:
title_to_sectionid['These Foolish Things']

[3866, 3867, 3868, 3869]

In [556]:
# take a section as input
s1 = 3868

#words = lines[s1].split()
print(tfidf[dictionary.doc2bow(processed_corpus[s1])])

[(0, 0.07770069015235205), (12, 0.025809626452381367), (97, 0.1256871903071076), (118, 0.12321934997480395), (120, 0.15333998889783754), (139, 0.06662649515930624), (197, 0.1593232218523629), (203, 0.06874600055109223), (260, 0.07953993222059288), (276, 0.1646580559616124), (342, 0.15182046621311437), (354, 0.14470811296851543), (417, 0.14109902631384016), (425, 0.1233693621836358), (459, 0.15303159840264635), (502, 0.25010438976246663), (733, 0.32774361914338773), (737, 0.27024244342052006), (804, 0.18027371709855108), (2283, 0.20671936030626425), (2371, 0.23223623358196308), (2373, 0.23660641187526477), (3671, 0.2757811026599757), (3672, 0.2757811026599757), (3676, 0.23660641187526477), (3678, 0.2757811026599757), (4625, 0.23660641187526477)]


In [557]:
from gensim import similarities

# Note: SparseMatrixSimilarity crashes due to lacking memory

index = similarities.Similarity('/content/', bow_corpus, num_features=len(dictionary))

2021-11-19 15:53:33,345 : INFO : starting similarity index under /content/


In [558]:
# take a tune as input
query = processed_corpus[s1]

query_bow = dictionary.doc2bow(query)
print(query_bow)

[(0, 1), (12, 1), (97, 1), (118, 1), (120, 1), (139, 1), (197, 1), (203, 1), (260, 1), (276, 1), (342, 1), (354, 1), (417, 1), (425, 1), (459, 1), (502, 1), (733, 1), (737, 1), (804, 1), (2283, 1), (2371, 1), (2373, 1), (3671, 1), (3672, 1), (3676, 1), (3678, 1), (4625, 1)]


In [559]:
tfidf[bow_corpus][s1]

[(0, 0.07770069015235205),
 (12, 0.025809626452381367),
 (97, 0.1256871903071076),
 (118, 0.12321934997480395),
 (120, 0.15333998889783754),
 (139, 0.06662649515930624),
 (197, 0.1593232218523629),
 (203, 0.06874600055109223),
 (260, 0.07953993222059288),
 (276, 0.1646580559616124),
 (342, 0.15182046621311437),
 (354, 0.14470811296851543),
 (417, 0.14109902631384016),
 (425, 0.1233693621836358),
 (459, 0.15303159840264635),
 (502, 0.25010438976246663),
 (733, 0.32774361914338773),
 (737, 0.27024244342052006),
 (804, 0.18027371709855108),
 (2283, 0.20671936030626425),
 (2371, 0.23223623358196308),
 (2373, 0.23660641187526477),
 (3671, 0.2757811026599757),
 (3672, 0.2757811026599757),
 (3676, 0.23660641187526477),
 (3678, 0.2757811026599757),
 (4625, 0.23660641187526477)]

In [560]:
doc = tfidf[bow_corpus][s1]
for id, freq in doc:
  print(dictionary[id], freq)

Am-D7 0.07770069015235205
Dm-G7 0.025809626452381367
B7-Em 0.1256871903071076
F#m7b5-B7 0.12321934997480395
F#m7b5-B7-Em 0.15333998889783754
D7-G7 0.06662649515930624
B7-Em-A7 0.1593232218523629
Em-A7 0.06874600055109223
A7-D7 0.07953993222059288
Em-A7-D7 0.1646580559616124
D7-G 0.15182046621311437
Am-D7-G7 0.14470811296851543
C#dim7-Dm 0.14109902631384016
Em-Am 0.1233693621836358
C#dim7-Dm-G7 0.15303159840264635
G7-C#dim7 0.25010438976246663
D7-G7-C#dim7 0.32774361914338773
G7-C#dim7-Dm 0.27024244342052006
Em-Am-D7 0.18027371709855108
G-Em 0.20671936030626425
D7-G-Em 0.23223623358196308
G-Em-Am 0.23660641187526477
C#m7b5-F#m7b5 0.2757811026599757
C#m7b5-F#m7b5-B7 0.2757811026599757
Em-C#m7b5 0.23660641187526477
Em-C#m7b5-F#m7b5 0.2757811026599757
A7-D7-G 0.23660641187526477


In [561]:
# perform a similarity query against the corpus
similarities = index[query_bow]
similarities

2021-11-19 15:53:34,782 : INFO : creating sparse index
2021-11-19 15:53:34,787 : INFO : creating sparse matrix from corpus
2021-11-19 15:53:34,796 : INFO : PROGRESS: at document #0/5403
2021-11-19 15:53:35,149 : INFO : created <5403x10302 sparse matrix of type '<class 'numpy.float32'>'
	with 117438 stored elements in Compressed Sparse Row format>
2021-11-19 15:53:35,150 : INFO : creating sparse shard #0
2021-11-19 15:53:35,156 : INFO : saving index shard to /content/.0
2021-11-19 15:53:35,160 : INFO : SparseMatrixSimilarity lifecycle event {'fname_or_handle': '/content/.0', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2021-11-19T15:53:35.160067', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'saving'}
2021-11-19 15:53:35,168 : INFO : saved /content/.0
2021-11-19 15:53:35,172 : INFO : loading SparseMatrixSimilarity object from /content/.0
202

array([0.07147416, 0.        , 0.04199605, ..., 0.06885304, 0.        ,
       0.03415437], dtype=float32)

In [562]:
sims = sorted(enumerate(similarities), key=lambda item: -item[1])

print(sectionid_to_section[s1])
print()
for s2, s2_score in sims[:20]:
    print(s2_score, sectionid_to_section[s2])

These Foolish Things, section3 (B)

0.99999976 These Foolish Things, section3 (B)
0.6543303 Isn't It A Pity, section3 (B)
0.5618006 More Than You Know, section3 (B)
0.51851857 Rosetta, section3 (B)
0.51851857 Rosetta, section3 (B)
0.5003192 Embraceable You, section2 (B)
0.4491477 If I Had You, section3 (B)
0.404226 They Can't Take That Away From Me, section3 (B)
0.40128618 Imagination, section3 (B)
0.39735967 All God's Chillun Got Rhythm, section2 (B)
0.39735967 Eiderdown, section4 (C)
0.39735967 Let's Get Lost, section2 (B)
0.3849002 Pure Imagination, section3 (B)
0.36115757 For Sentimental Reasons, section3 (B)
0.36115757 I've Told Eve'ry Little Star, section3 (B)
0.3603933 Beyond The Sea, section3 (B)
0.35737085 Folks Who Live On The Hill, The, section3 (B)
0.3478328 Walkin' My Baby Back Home, section4 (B)
0.34565058 At Last, section3 (B)
0.33898345 Crazy He Calls Me, section3 (B)


## Test for single tunes

In [563]:
tunes_eval_list = [
  'Sweet Sue, Just You',
  'These Foolish Things', 
  'Blue Moon',
  'All Of Me',
  "All God's Chillun Got Rhythm",
  'I Got Rhythm',
  'Bye Bye Blackbird',
  'Old Fashioned Love',
  'Dinah',
  'Honeysuckle Rose',
  'Misty'
]

Get the most similar documents using the TF-IDF model.

In [564]:
df_sim = pd.DataFrame(columns=['reference_title',
                               'similar_title',
                               'ref_section', 
                               'similar_section', 
                               'score', 
                               ])

for tune in tunes_eval_list:
  print()
  print("-"*50)
  for s1 in title_to_sectionid[tune]:
  
      query = processed_corpus[s1]
      query_bow = dictionary.doc2bow(query)

      # perform a similarity query against the corpus
      similarities = index[query_bow]
      sims = sorted(enumerate(similarities), key=lambda item: -item[1])
      
      print(s1, sectionid_to_section[s1])
      n = 0
      for s2, s2_score in sims:
        
        # store the top N best results
        if n > 40:
            break
        # don't count self-similarity between sections of the same tune
        if s2 not in title_to_sectionid[tune]:
            n += 1
        
            # print(f"\t{s2_score:.3f} {sectionid_to_section[s2]}")

            df_sim.loc[len(df_sim)] = [tune,
                            sectionid_to_title[s2],
                            sectionid_to_section[s1], 
                            sectionid_to_section[s2], 
                            s2_score, 
                            ]

     


--------------------------------------------------
3752 Sweet Sue, Just You, section1 (A)
3753 Sweet Sue, Just You, section2 (A)
3754 Sweet Sue, Just You, section3 (B)
3755 Sweet Sue, Just You, section4 (A)

--------------------------------------------------
3866 These Foolish Things, section1 (A)
3867 These Foolish Things, section2 (A)
3868 These Foolish Things, section3 (B)
3869 These Foolish Things, section4 (A)

--------------------------------------------------
496 Blue Moon, section1 (A)
497 Blue Moon, section2 (A)
498 Blue Moon, section3 (B)
499 Blue Moon, section4 (A)

--------------------------------------------------
198 All Of Me, section1 (A)
199 All Of Me, section2 (B)
200 All Of Me, section3 (A)
201 All Of Me, section4 (C)

--------------------------------------------------
188 All God's Chillun Got Rhythm, section1 (A)
189 All God's Chillun Got Rhythm, section2 (B)
190 All God's Chillun Got Rhythm, section3 (A)
191 All God's Chillun Got Rhythm, section4 (C)

-----------

In [565]:
from google.colab import files

file = 'tfidf_results.csv'
df_sim.to_csv(file)
files.download(file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [566]:
df_sim.head(50)

,reference_title,similar_title,ref_section,similar_section,score
0,"Sweet Sue, Just You",Get Me To The Church On Time,"Sweet Sue, Just You, section1 (A)","Get Me To The Church On Time, section1 (A)",0.943636
1,"Sweet Sue, Just You",Get Me To The Church On Time,"Sweet Sue, Just You, section1 (A)","Get Me To The Church On Time, section2 (A)",0.943636
2,"Sweet Sue, Just You",My Secret Love,"Sweet Sue, Just You, section1 (A)","My Secret Love, section2 (A)",0.927279
3,"Sweet Sue, Just You",Don't Fence Me In,"Sweet Sue, Just You, section1 (A)","Don't Fence Me In, section1 (A)",0.906202
4,"Sweet Sue, Just You",Samba De Orfeu,"Sweet Sue, Just You, section1 (A)","Samba De Orfeu, section2 (A)",0.906202
5,"Sweet Sue, Just You",Brazil (Aquarela Do Brasil),"Sweet Sue, Just You, section1 (A)","Brazil (Aquarela Do Brasil), section1 (verse)",0.885543
6,"Sweet Sue, Just You",It's De-lovely,"Sweet Sue, Just You, section1 (A)","It's De-lovely, section3 (A)",0.879343
7,"Sweet Sue, Just You",My Secret Love,"Sweet Sue, Just You, section1 (A)","My Secret Love, section1 (A)",0.871849
8,"Sweet Sue, Just You",That Old Black Magic,"Sweet Sue, Just You, section1 (A)","That Old Black Magic, section1 (A)",0.862913
9,"Sweet Sue, Just You",Down By The Riverside,"Sweet Sue, Just You, section1 (A)","Down By The Riverside, section1 (A)",0.848493


In [567]:
import plotly.express as px
fig = px.histogram(df_sim, x="score", nbins=100)
fig.show()

Note: Testing Self-Similarity for TF-IDF model does not make sense, since it will always be 1.0.

### Calculate a Recommender Score for a Specific Tune

In [585]:
tune_name = 'These Foolish Things'

df_tune = df_sim.query(f"reference_title == '{tune_name}'")

ff = df_tune.iloc[:]

# get the maximum similarity score for each section and store in new column
ff['max'] = ff.groupby('ref_section')['score'].transform('max')

# scale the score with the maxmum value of each section
ff['score_div_max'] = ff['score'] / ff['max']

# consider only the top N tunes for each group
result = ff.sort_values(['ref_section', 'score_div_max'], ascending=[True, False])
result = result.groupby('ref_section').head(20)
result.head(50)

,reference_title,similar_title,ref_section,similar_section,score,max,score_div_max
164,These Foolish Things,"Way You Look Tonight, The","These Foolish Things, section1 (A)","Way You Look Tonight, The, section1 (A)",0.820646,0.820646,1.000000
165,These Foolish Things,I'm Old Fashioned,"These Foolish Things, section1 (A)","I'm Old Fashioned, section3 (C)",0.789057,0.820646,0.961507
166,These Foolish Things,I Won't Dance,"These Foolish Things, section1 (A)","I Won't Dance, section1 (A)",0.789022,0.820646,0.961465
167,These Foolish Things,Mountain Greenery,"These Foolish Things, section1 (A)","Mountain Greenery, section4 (A)",0.781358,0.820646,0.952125
168,These Foolish Things,Heart And Soul,"These Foolish Things, section1 (A)","Heart And Soul, section1 (A)",0.769989,0.820646,0.938272
169,These Foolish Things,Let's Fall In Love,"These Foolish Things, section1 (A)","Let's Fall In Love, section4 (A)",0.769989,0.820646,0.938272
170,These Foolish Things,Heart And Soul,"These Foolish Things, section1 (A)","Heart And Soul, section4 (A)",0.766334,0.820646,0.933818
171,These Foolish Things,"Way You Look Tonight, The","These Foolish Things, section1 (A)","Way You Look Tonight, The, section2 (A)",0.764668,0.820646,0.931788
172,These Foolish Things,"Touch Of Your Lips, The","These Foolish Things, section1 (A)","Touch Of Your Lips, The, section1 (A)",0.761492,0.820646,0.927918
173,These Foolish Things,"Touch Of Your Lips, The","These Foolish Things, section1 (A)","Touch Of Your Lips, The, section3 (A)",0.761492,0.820646,0.927918


In [586]:
result.query('ref_section == "These Foolish Things, section3 (B)"')

,reference_title,similar_title,ref_section,similar_section,score,max,score_div_max
246,These Foolish Things,Isn't It A Pity,"These Foolish Things, section3 (B)","Isn't It A Pity, section3 (B)",0.654330,0.65433,1.000000
247,These Foolish Things,More Than You Know,"These Foolish Things, section3 (B)","More Than You Know, section3 (B)",0.561801,0.65433,0.858589
248,These Foolish Things,Rosetta,"These Foolish Things, section3 (B)","Rosetta, section3 (B)",0.518519,0.65433,0.792442
249,These Foolish Things,Rosetta,"These Foolish Things, section3 (B)","Rosetta, section3 (B)",0.518519,0.65433,0.792442
250,These Foolish Things,Embraceable You,"These Foolish Things, section3 (B)","Embraceable You, section2 (B)",0.500319,0.65433,0.764628
251,These Foolish Things,If I Had You,"These Foolish Things, section3 (B)","If I Had You, section3 (B)",0.449148,0.65433,0.686423
252,These Foolish Things,They Can't Take That Away From Me,"These Foolish Things, section3 (B)","They Can't Take That Away From Me, section3 (B)",0.404226,0.65433,0.617771
253,These Foolish Things,Imagination,"These Foolish Things, section3 (B)","Imagination, section3 (B)",0.401286,0.65433,0.613278
254,These Foolish Things,All God's Chillun Got Rhythm,"These Foolish Things, section3 (B)","All God's Chillun Got Rhythm, section2 (B)",0.397360,0.65433,0.607277
255,These Foolish Things,Eiderdown,"These Foolish Things, section3 (B)","Eiderdown, section4 (C)",0.397360,0.65433,0.607277


In [587]:
# keep only the simmilar_title with the highest score_div_max
result = result.groupby('similar_title').max('score_div_max').sort_values('score_div_max', ascending=False)
result.head(20)

,score,max,score_div_max
similar_title,,,
"Way You Look Tonight, The",0.820646,0.820646,1.000000
Isn't It A Pity,0.654330,0.654330,1.000000
I Like The Likes Of You,0.809040,0.812277,0.996015
I Won't Dance,0.789022,0.820646,0.981003
Mountain Greenery,0.781358,0.820646,0.978787
I'm Old Fashioned,0.789057,0.820646,0.961507
"Party's Over, The",0.779222,0.812277,0.959305
Too Young To Go Steady,0.733333,0.771930,0.950000
Heart And Soul,0.769989,0.820646,0.946878


### Put the Recommendation in a Function

Input: `df_sim` from above

In [588]:
def recommend_tune(df, tune_name):

  df_tune = df.query(f"reference_title == '{tune_name}'")
  ff = df_tune.iloc[:]

  # get the maximum similarity score for each section and store in new column
  ff['max'] = ff.groupby('ref_section')['score'].transform('max')

  # scale the score with the maxmum value of each section
  ff['score_div_max'] = ff['score'] / ff['max']

  # consider only the top N tunes for each group
  result = ff.sort_values(['ref_section', 'score_div_max'], ascending=[True, False])
  result = result.groupby('ref_section').head(30)
  
  # keep only the similar_title with the highest score_div_max
  result = result.groupby('similar_title').max('score_div_max').sort_values('score_div_max', ascending=False)

  return result

In [589]:
recommend_tune(df_sim, 'These Foolish Things').head(30)

,score,max,score_div_max
similar_title,,,
"Way You Look Tonight, The",0.820646,0.820646,1.000000
Isn't It A Pity,0.654330,0.654330,1.000000
I Like The Likes Of You,0.809040,0.812277,0.996015
I Won't Dance,0.789022,0.820646,0.981003
Mountain Greenery,0.781358,0.820646,0.978787
I'm Old Fashioned,0.789057,0.820646,0.961507
"Party's Over, The",0.779222,0.820646,0.959305
Too Young To Go Steady,0.733333,0.771930,0.950000
Heart And Soul,0.769989,0.820646,0.946878


## Test of the Contrafacts tunes

`contrafacts_test` provides a list of the section ids for two tunes who are under test. Here are a few examples:

In [572]:
for tune1, tune2 in contrafacts_test[:3]:
  print(f"{tune1}, {tune2}")

[0, 1, 2, 3], [810, 811, 812, 813]
[6, 7, 8, 9], [1647, 1648, 1649, 1650, 4821, 4822, 4823, 4824]
[104, 105, 106, 107], [209, 210, 211, 212]


Calculate the cosinus similarity between each section using the TF-IDF model.

In [573]:
# create the empty dataframe for the results, with a column for each ngram value
cols = ['Tune1', 'Tune2', 'Section1', 'Section2', 'Score']
df_tfidf = pd.DataFrame(columns=cols)

for tune1, tune2 in contrafacts_test:
    tune_sim = []
    # loop over all combinations of sections
    for s1 in tune1:
        for s2 in tune2:
            tfidf1 = tfidf[dictionary.doc2bow(processed_corpus[s1])]
            tfidf2 = tfidf[dictionary.doc2bow(processed_corpus[s2])]
            score = cosine_similarity(tfidf1, tfidf2)[0][0]

            row = [sectionid_to_title[s1],
                   sectionid_to_title[s2],
                   sectionid_to_section[s1], 
                   sectionid_to_section[s2], 
                   score]
            df_tfidf.loc[len(df_tfidf)] = row

In [574]:
df_tfidf.sort_values(['Tune1', 'Score'], ascending=[True, False]).head(50)

,Tune1,Tune2,Section1,Section2,Score
0,26-2,Confirmation,"26-2, section1 (A)","Confirmation, section1 (A)",1.000000
1,26-2,Confirmation,"26-2, section1 (A)","Confirmation, section2 (A)",1.000000
3,26-2,Confirmation,"26-2, section1 (A)","Confirmation, section4 (A)",1.000000
10,26-2,Confirmation,"26-2, section3 (B)","Confirmation, section3 (B)",0.999997
6,26-2,Confirmation,"26-2, section2 (A)","Confirmation, section3 (B)",0.994904
14,26-2,Confirmation,"26-2, section4 (A)","Confirmation, section3 (B)",0.994904
4,26-2,Confirmation,"26-2, section2 (A)","Confirmation, section1 (A)",0.111133
5,26-2,Confirmation,"26-2, section2 (A)","Confirmation, section2 (A)",0.111133
7,26-2,Confirmation,"26-2, section2 (A)","Confirmation, section4 (A)",0.111133
12,26-2,Confirmation,"26-2, section4 (A)","Confirmation, section1 (A)",0.111133


We can see in the histogram of the Score, that the match beween the sections is either very good or very bad.

In [575]:
import plotly.express as px
fig = px.histogram(df_tfidf, x="Score", nbins=100)
fig.show()

In [576]:
#df_tfidf_pivot = pd.pivot_table(df_tfidf, 
#                                index = ['Tune1', 'Tune2', 'Section1', 'Section2'], 
#                                #columns = 'ngrams', 
#                                values = 'Score')
#df_tfidf_pivot.head(50)

# SVD / LSI Example

## Use various ngrams as input

In [146]:
ngrams_for_input = [1, 2]

data_ngrams = []
for line in data:
  tune_n = []
  for n in ngrams_for_input:
    tune_n.extend(ngrams(line, n=n))
  data_ngrams.append(tune_n)

for line in data_ngrams[:5]:
  print(line)

['C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm', 'C7', 'F', 'G#7', 'C#', 'E7', 'Am', 'D7', 'Dm', 'G7', 'C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm', 'Gm-C7', 'C7-F', 'F-G#7', 'G#7-C#', 'C#-E7', 'E7-Am', 'Am-D7', 'D7-Dm', 'Dm-G7']
['C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C', 'C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm', 'Gm-C7', 'C7-F', 'F-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-C']
['Gm', 'C7', 'Bm', 'E7', 'A', 'C7', 'F', 'Bbm', 'Eb7', 'G#', 'Dm', 'G7', 'Gm-C7', 'C7-Bm', 'Bm-E7', 'E7-A', 'A-C7', 'C7-F', 'F-Bbm', 'Bbm-Eb7', 'Eb7-G#', 'G#-Dm', 'Dm-G7']
['C', 'Eb7', 'G#', 'B7', 'E', 'G7', 'Gm', 'C7', 'F', 'Eb7', 'G#', 'B7', 'E', 'G7', 'C', 'C-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-Gm', 'Gm-C7', 'C7-F', 'F-Eb7', 'Eb7-G#', 'G#-B7', 'B7-E', 'E-G7', 'G7-C']
['Am', 'Am', 'Cm', 'Cm', 'Eb', 'Eb', 'Em7b5', 'A7', 'Dm', 'Dm', 'Bm7b5', 'Bm7b5', 'Bbm', 'Bbm', 'Fm', 'Fm', 'E7', 'E7', 'Fm', 'Fm', 'C#', 'C#', 'Fm', 'Fm', 'C#', 'C#', 'Am-Am', '

In [147]:
from gensim import corpora, models, similarities
from collections import defaultdict

frequency = defaultdict(int)
for text in data_ngrams:
    for token in text:
        frequency[token] += 1

data_ngrams = [[token for token in text if frequency[token] > 1] for text in data_ngrams]
dictionary = corpora.Dictionary(data_ngrams)

# doc2bow counts the number of occurences of each distinct word,
# converts the word to its integer word id and returns the result
# as a sparse vector

corpus = [dictionary.doc2bow(text) for text in data_ngrams]
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=200)  # num_topics can be maximum the size of the number of unique tokens

2021-11-19 10:20:04,078 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-11-19 10:20:04,312 : INFO : built Dictionary(1602 unique tokens: ['Am', 'Am-D7', 'B7', 'B7-E', 'C']...) from 5403 documents (total 159684 corpus positions)
2021-11-19 10:20:04,314 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(1602 unique tokens: ['Am', 'Am-D7', 'B7', 'B7-E', 'C']...) from 5403 documents (total 159684 corpus positions)", 'datetime': '2021-11-19T10:20:04.314896', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'created'}
2021-11-19 10:20:04,438 : INFO : using serial LSI version on this node
2021-11-19 10:20:04,439 : INFO : updating model with new documents
2021-11-19 10:20:04,442 : INFO : preparing a new chunk of documents
2021-11-19 10:20:04,489 : INFO : using 100 extra samples and 2 power iterations
2021-11-19 10:20:04,494 : INFO : 1st phase: constructing 

In [148]:
lsi.get_topics().shape

(200, 1602)

In [149]:
title_to_sectionid['These Foolish Things']

[3866, 3867, 3868, 3869]

In [150]:
# add new sample

i = 3866
print(f"Check similarity for {sectionid_to_section[i]}")
tune = data_ngrams[i]
vec_bow = dictionary.doc2bow(tune)

# convert the query to LSI space
vec_lsi = lsi[vec_bow]
index = similarities.MatrixSimilarity(lsi[corpus])
print(len(vec_lsi))

2021-11-19 10:20:17,335 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)


Check similarity for These Foolish Things, section1 (A)


2021-11-19 10:20:17,889 : INFO : creating matrix with 5403 documents and 200 features


200


In [151]:
# perform a similarity query against the corpus
sims = index[vec_lsi]
sims = sorted(enumerate(sims), key=lambda item: -item[1])

print(sims)
print(len(sims))

[(3866, 1.0), (3869, 0.96119756), (4142, 0.9178291), (3867, 0.9162063), (4143, 0.89279896), (1487, 0.8849038), (2341, 0.88308704), (1777, 0.8780154), (3870, 0.8743566), (3109, 0.8735132), (420, 0.87124467), (3485, 0.87075406), (312, 0.86702657), (2774, 0.8638803), (1485, 0.8633582), (500, 0.8633413), (4531, 0.8633413), (2449, 0.86304915), (2451, 0.86304915), (3086, 0.86024994), (3146, 0.85754335), (2776, 0.8566006), (1286, 0.85647476), (4884, 0.8552995), (423, 0.85527056), (1484, 0.85493135), (2344, 0.85493135), (2773, 0.8534633), (226, 0.85313034), (2808, 0.85313034), (3989, 0.8526864), (3991, 0.8526864), (1846, 0.8525993), (1394, 0.85138667), (2887, 0.85060894), (1042, 0.850137), (835, 0.8497112), (501, 0.849382), (4532, 0.849382), (3983, 0.84781486), (1472, 0.8475907), (2153, 0.84695315), (2154, 0.84695315), (503, 0.84632283), (4534, 0.84632283), (2088, 0.8463204), (496, 0.845092), (499, 0.845092), (1474, 0.8447776), (1299, 0.8394623), (4251, 0.8388419), (1000, 0.83599645), (1285, 0

In [152]:
print(f"Similarity for '{sectionid_to_section[i]}'")
print()

for id, sim in sims[:15]:
  print(f"{sim:.3f} {sectionid_to_section[id]}")

Similarity for 'These Foolish Things, section1 (A)'

1.000 These Foolish Things, section1 (A)
0.961 These Foolish Things, section4 (A)
0.918 Way You Look Tonight, The, section1 (A)
0.916 These Foolish Things, section2 (A)
0.893 Way You Look Tonight, The, section2 (A)
0.885 Heart And Soul, section4 (A)
0.883 Let's Fall In Love, section1 (A)
0.878 I Won't Dance, section1 (A)
0.874 They All Laughed, section1 (A)
0.874 Party's Over, The, section2 (B)
0.871 Bewitched, section1 (A)
0.871 Smoke Gets In Your Eyes, section4 (A)
0.867 At Last, section4 (A)
0.864 Mountain Greenery, section2 (A)
0.863 Heart And Soul, section2 (A)


#### Test for single tunes

In [153]:
# test a sample

n = 4

for tune in tunes_eval_list:
  print(f"\n\n*** {tune} ***")
  for s1 in title_to_sectionid[tune]:

    print(f"\nCheck similarity for {sectionid_to_section[s1]}")
    tune = data_ngrams[s1]
    vec_bow = dictionary.doc2bow(tune)

    # convert the query to LSI space
    vec_lsi = lsi[vec_bow]
    index = similarities.MatrixSimilarity(lsi[corpus])

    # perform a similarity query against the corpus
    sims = index[vec_lsi]
    sims = sorted(enumerate(sims), key=lambda item: -item[1])

    for id, sim in sims[:15]:
      print(f"{sim:.3f} {sectionid_to_section[id]}")

2021-11-19 10:20:21,065 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)




*** Sweet Sue, Just You ***

Check similarity for Sweet Sue, Just You, section1 (A)


2021-11-19 10:20:21,604 : INFO : creating matrix with 5403 documents and 200 features
2021-11-19 10:20:22,685 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)


1.000 Sweet Sue, Just You, section1 (A)
1.000 Sweet Sue, Just You, section2 (A)
1.000 Sweet Sue, Just You, section4 (A)
0.979 My Secret Love, section2 (A)
0.975 I Want To Be Happy, section2 (A)
0.975 I Want To Be Happy, section4 (A)
0.972 Don't Fence Me In, section1 (A)
0.970 Get Me To The Church On Time, section1 (A)
0.970 Get Me To The Church On Time, section2 (A)
0.968 It's De-lovely, section3 (A)
0.965 Samba De Orfeu, section2 (A)
0.963 Just One More Chance, section1 (A)
0.962 Cherry Pink And Apple Blossom White, section1 (A)
0.962 Same Old Love, The, section1 (A)
0.955 La Vie En Rose, section1 (A)

Check similarity for Sweet Sue, Just You, section2 (A)


2021-11-19 10:20:23,237 : INFO : creating matrix with 5403 documents and 200 features
2021-11-19 10:20:24,301 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)


...

1.000 Misty, section2 (A)
0.917 Misty, section4 (A)
0.878 I Won't Dance, section2 (A)
0.855 Anything Goes, section1 (A)
0.851 S'posin', section3 (A)
0.846 Glory Of Love, The, section2 (A)
0.846 My Way, section1 (A)
0.845 I've Got You Under My Skin, section4 (D)
0.835 It's A Lovely Day Today, section1 (A)
0.835 It's A Lovely Day Today, section3 (A)
0.833 You Turned The Tables on Me, section2 (B)
0.833 My Way, section2 (A)
0.826 Could It Be You, section2 (A)
0.826 I've Got My Fingers Crossed, section2 (A)
0.826 Easy Living, section4 (A)

Check similarity for Misty, section3 (B)


2021-11-19 10:21:10,501 : INFO : creating matrix with 5403 documents and 200 features
2021-11-19 10:21:11,552 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)


1.000 Misty, section3 (B)
0.772 Hocus-Pocus, section3 (B)
0.754 Eclypso, section3 (B)
0.747 Everything Happens To Me, section3 (B)
0.730 Move, section3 (B)
0.730 You Can Depend On Me, section3 (B)
0.730 You Can Depend On Me, section3 (B)
0.722 I'm Confessin' That I Love You, section3 (B)
0.722 On The Sunny Side Of The Street, section3 (B)
0.722 On The Sunny Side Of The Street, section3 (B)
0.717 Nevertheless, section3 (B)
0.707 Suddenly It's Spring, section2 (B)
0.703 I Guess I'll Hang My Tears Out To Dry, section3 (B)
0.697 I Can't Give You Anything But Love, section2 (B)
0.697 I Can't Give You Anything But Love, section2 (B)

Check similarity for Misty, section4 (A)


2021-11-19 10:21:12,104 : INFO : creating matrix with 5403 documents and 200 features


1.000 Misty, section4 (A)
0.917 Misty, section2 (A)
0.896 I Won't Dance, section2 (A)
0.893 Portrait Of Jennie, section4 (A)
0.893 It's A Lovely Day Today, section1 (A)
0.893 It's A Lovely Day Today, section3 (A)
0.889 Misty, section1 (A)
0.888 Long Ago And Far Away, section4 (C)
0.883 Why Do I Love You, section2 (A)
0.876 Time After Time, section2 (B)
0.876 I've Got You Under My Skin, section4 (D)
0.872 Way You Look Tonight, The, section2 (A)
0.871 Could It Be You, section2 (A)
0.870 I'm Putting All My Eggs In One Basket, section1 (A)
0.870 I'm Putting All My Eggs In One Basket, section4 (A)


# Doc2Vec Embeddings

## Read and Tag Data

In [ ]:
df.head()

,id,file_name,title,tune_mode,tune_id,section_name,section_id,chords
0,0,dataset/jazz1350/26-2.xml,26-2,major,0,A,1,CM7 Eb7 G#M7 B7 EM7 G7 Gm7 C7 FM7 G#7 C#M7 E7 ...
1,1,dataset/jazz1350/26-2.xml,26-2,major,0,A,2,CM7 Eb7 G#M7 B7 EM7 G7 Gm7 C7 FM7 Eb7 G#M7 B7 ...
2,2,dataset/jazz1350/26-2.xml,26-2,major,0,B,3,Gm7 C7 Bm7 E7 AM7 C7 FM7 Bbm7 Eb7 G#M7 Dm7 G7
3,3,dataset/jazz1350/26-2.xml,26-2,major,0,A,4,CM7 Eb7 G#M7 B7 EM7 G7 Gm7 C7 FM7 Eb7 G#M7 B7 ...
4,4,dataset/jazz1350/500 Miles High.xml,500 Miles High,minor,1,NaN,0,Am7 Am7 Cm7 Cm7 EbM7 EbM7 Em7b5 A7 Dm7 Dm7 Bm7...


In [ ]:

for i, row in df.iterrows():
  tokens = []
  for ngram in range(3):
    tokens.extend(ngrams(row['chords'].split(), ngram, '-'))
  


[[], ['CM7', 'Eb7', 'G#M7', 'B7', 'EM7', 'G7', 'Gm7', 'C7', 'FM7', 'G#7', 'C#M7', 'E7', 'Am7', 'D7', 'Dm7', 'G7'], ['CM7-Eb7', 'Eb7-G#M7', 'G#M7-B7', 'B7-EM7', 'EM7-G7', 'G7-Gm7', 'Gm7-C7', 'C7-FM7', 'FM7-G#7', 'G#7-C#M7', 'C#M7-E7', 'E7-Am7', 'Am7-D7', 'D7-Dm7', 'Dm7-G7'], [], ['CM7', 'Eb7', 'G#M7', 'B7', 'EM7', 'G7', 'Gm7', 'C7', 'FM7', 'Eb7', 'G#M7', 'B7', 'EM7', 'G7', 'CM7'], ['CM7-Eb7', 'Eb7-G#M7', 'G#M7-B7', 'B7-EM7', 'EM7-G7', 'G7-Gm7', 'Gm7-C7', 'C7-FM7', 'FM7-Eb7', 'Eb7-G#M7', 'G#M7-B7', 'B7-EM7', 'EM7-G7', 'G7-CM7'], [], ['Gm7', 'C7', 'Bm7', 'E7', 'AM7', 'C7', 'FM7', 'Bbm7', 'Eb7', 'G#M7', 'Dm7', 'G7'], ['Gm7-C7', 'C7-Bm7', 'Bm7-E7', 'E7-AM7', 'AM7-C7', 'C7-FM7', 'FM7-Bbm7', 'Bbm7-Eb7', 'Eb7-G#M7', 'G#M7-Dm7', 'Dm7-G7'], [], ['CM7', 'Eb7', 'G#M7', 'B7', 'EM7', 'G7', 'Gm7', 'C7', 'FM7', 'Eb7', 'G#M7', 'B7', 'EM7', 'G7', 'CM7'], ['CM7-Eb7', 'Eb7-G#M7', 'G#M7-B7', 'B7-EM7', 'EM7-G7', 'G7-Gm7', 'Gm7-C7', 'C7-FM7', 'FM7-Eb7', 'Eb7-G#M7', 'G#M7-B7', 'B7-EM7', 'EM7-G7', 'G7-CM7'], [

In [ ]:
def read_corpus(df, max_ngram_size, tokens_only=False):
    # concatenate unigrams, bigrams and trigrams and use them as input; use PV-DBOW since it does not depend on the order
    for i, row in df.iterrows():
      tokens = []
      for ngram in range(max_ngram_size + 1):
        tokens.extend(ngrams(row['chords'].split(), ngram, '-'))
      if tokens_only:
          yield tokens
      else:
          # for training data, add tags
          yield gensim.models.doc2vec.TaggedDocument(tokens, [row['id'], row['tune_id']])


## Doc2Vec Hyperparameters

In [ ]:
!rm -R models_doc2vec
!rm -R traindata_doc2vec

In [ ]:
!mkdir models_doc2vec
!mkdir traindata_doc2vec

In [ ]:
import multiprocessing

model_path = './models_doc2vec'
model_prefix = 'autos'
data_path = './traindata_doc2vec'

param_grid = {'d2v': {'variant': [#'pv-dm', 'concat',
                                  'pv-dbow'
                                  ], 
                      'window': [
                                 #2, 
                                 3, 
                                 4, 
                                 10
                                 ],
                      'ngrams': [
                                 #1, 
                                 #2, 
                                 3, 
                                 #4
                                 ],
                     }
              }


common_kwargs = dict(
    vector_size=100, 
    epochs=40, 
    min_count=1,
    sample=0, 
    workers=multiprocessing.cpu_count(), 
    negative=10, 
    hs=0,
)

for algo, params in param_grid.items():
    for variant in params['variant']:
        dm = 0 if variant == 'pv-dbow' else 1
        concat = 1 if variant ==  'concat' else 0

        for ngram_size in params['ngrams']:
            
            # prepare data with the provided ngram size
            train_corpus = list(read_corpus(df, max_ngram_size=3))

            # evaluate different window sizes
            for window in params['window']:

                # calculate the model
                if algo == 'd2v':
                    print(f"Model: {variant}, dm={dm}, dm_concat={concat}; ngrams={ngram_size}, window={window}")
                    print(f"Corpus: {train_corpus[0]}")
                    model = gensim.models.doc2vec.Doc2Vec(**common_kwargs,
                                                          dm = dm,
                                                          dm_concat = concat,
                                                          )
                    model.build_vocab(train_corpus)
                    model.train(train_corpus, 
                                total_examples=model.corpus_count, 
                                epochs=model.epochs)

                else:
                    model = gensim.models.FastText(data_ngrams, vector_size=size, window=window, sg=sg, epochs=50)

                file_name = f"{model_path}/{model_prefix}_{algo}_{variant}_{ngram_size}_{window}"
                model.save(file_name + '.model')

                file_name = f"{data_path}/{model_prefix}_{algo}_{variant}_{ngram_size}_{window}"
                with open(file_name + '.train', 'wb') as f:
                      # Pickle the 'data' dictionary using the highest protocol available.
                      pickle.dump(train_corpus, f, pickle.HIGHEST_PROTOCOL)

2021-11-18 20:20:15,148 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dbow,d100,n10,t2)', 'datetime': '2021-11-18T20:20:15.148491', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'created'}
2021-11-18 20:20:15,150 : INFO : collecting all words and their counts
2021-11-18 20:20:15,156 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2021-11-18 20:20:15,224 : INFO : collected 15908 word types and 5403 unique tags from a corpus of 5403 examples and 232161 words
2021-11-18 20:20:15,226 : INFO : Creating a fresh vocabulary
2021-11-18 20:20:15,304 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=1 retains 15908 unique words (100.0%% of original 15908, drops 0)', 'datetime': '2021-11-18T20:20:15.304718', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu

Model: pv-dbow, dm=0, dm_concat=0; ngrams=3, window=3
Corpus: TaggedDocument(['CM7', 'Eb7', 'G#M7', 'B7', 'EM7', 'G7', 'Gm7', 'C7', 'FM7', 'G#7', 'C#M7', 'E7', 'Am7', 'D7', 'Dm7', 'G7', 'CM7-Eb7', 'Eb7-G#M7', 'G#M7-B7', 'B7-EM7', 'EM7-G7', 'G7-Gm7', 'Gm7-C7', 'C7-FM7', 'FM7-G#7', 'G#7-C#M7', 'C#M7-E7', 'E7-Am7', 'Am7-D7', 'D7-Dm7', 'Dm7-G7', 'CM7-Eb7-G#M7', 'Eb7-G#M7-B7', 'G#M7-B7-EM7', 'B7-EM7-G7', 'EM7-G7-Gm7', 'G7-Gm7-C7', 'Gm7-C7-FM7', 'C7-FM7-G#7', 'FM7-G#7-C#M7', 'G#7-C#M7-E7', 'C#M7-E7-Am7', 'E7-Am7-D7', 'Am7-D7-Dm7', 'D7-Dm7-G7'], [0, 0])


2021-11-18 20:20:15,428 : INFO : deleting the raw counts dictionary of 15908 items
2021-11-18 20:20:15,432 : INFO : sample=0 downsamples 0 most-common words
2021-11-18 20:20:15,435 : INFO : Doc2Vec lifecycle event {'msg': 'downsampling leaves estimated 232161 word corpus (100.0%% of prior 232161)', 'datetime': '2021-11-18T20:20:15.435115', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'prepare_vocab'}
2021-11-18 20:20:15,651 : INFO : estimated required memory for 15908 words and 100 dimensions: 23922200 bytes
2021-11-18 20:20:15,654 : INFO : resetting layer weights
2021-11-18 20:20:15,673 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 2 workers on 15908 vocabulary and 100 features, using sg=1 hs=0 sample=0 negative=10 window=5 shrink_windows=True', 'datetime': '2021-11-18T20:20:15.673447', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[

Model: pv-dbow, dm=0, dm_concat=0; ngrams=3, window=4
Corpus: TaggedDocument(['CM7', 'Eb7', 'G#M7', 'B7', 'EM7', 'G7', 'Gm7', 'C7', 'FM7', 'G#7', 'C#M7', 'E7', 'Am7', 'D7', 'Dm7', 'G7', 'CM7-Eb7', 'Eb7-G#M7', 'G#M7-B7', 'B7-EM7', 'EM7-G7', 'G7-Gm7', 'Gm7-C7', 'C7-FM7', 'FM7-G#7', 'G#7-C#M7', 'C#M7-E7', 'E7-Am7', 'Am7-D7', 'D7-Dm7', 'Dm7-G7', 'CM7-Eb7-G#M7', 'Eb7-G#M7-B7', 'G#M7-B7-EM7', 'B7-EM7-G7', 'EM7-G7-Gm7', 'G7-Gm7-C7', 'Gm7-C7-FM7', 'C7-FM7-G#7', 'FM7-G#7-C#M7', 'G#7-C#M7-E7', 'C#M7-E7-Am7', 'E7-Am7-D7', 'Am7-D7-Dm7', 'D7-Dm7-G7'], [0, 0])


2021-11-18 20:21:05,811 : INFO : deleting the raw counts dictionary of 15908 items
2021-11-18 20:21:05,813 : INFO : sample=0 downsamples 0 most-common words
2021-11-18 20:21:05,815 : INFO : Doc2Vec lifecycle event {'msg': 'downsampling leaves estimated 232161 word corpus (100.0%% of prior 232161)', 'datetime': '2021-11-18T20:21:05.815335', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'prepare_vocab'}
2021-11-18 20:21:06,025 : INFO : estimated required memory for 15908 words and 100 dimensions: 23922200 bytes
2021-11-18 20:21:06,027 : INFO : resetting layer weights
2021-11-18 20:21:06,048 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 2 workers on 15908 vocabulary and 100 features, using sg=1 hs=0 sample=0 negative=10 window=5 shrink_windows=True', 'datetime': '2021-11-18T20:21:06.048543', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[

Model: pv-dbow, dm=0, dm_concat=0; ngrams=3, window=10
Corpus: TaggedDocument(['CM7', 'Eb7', 'G#M7', 'B7', 'EM7', 'G7', 'Gm7', 'C7', 'FM7', 'G#7', 'C#M7', 'E7', 'Am7', 'D7', 'Dm7', 'G7', 'CM7-Eb7', 'Eb7-G#M7', 'G#M7-B7', 'B7-EM7', 'EM7-G7', 'G7-Gm7', 'Gm7-C7', 'C7-FM7', 'FM7-G#7', 'G#7-C#M7', 'C#M7-E7', 'E7-Am7', 'Am7-D7', 'D7-Dm7', 'Dm7-G7', 'CM7-Eb7-G#M7', 'Eb7-G#M7-B7', 'G#M7-B7-EM7', 'B7-EM7-G7', 'EM7-G7-Gm7', 'G7-Gm7-C7', 'Gm7-C7-FM7', 'C7-FM7-G#7', 'FM7-G#7-C#M7', 'G#7-C#M7-E7', 'C#M7-E7-Am7', 'E7-Am7-D7', 'Am7-D7-Dm7', 'D7-Dm7-G7'], [0, 0])


2021-11-18 20:21:52,764 : INFO : deleting the raw counts dictionary of 15908 items
2021-11-18 20:21:52,767 : INFO : sample=0 downsamples 0 most-common words
2021-11-18 20:21:52,768 : INFO : Doc2Vec lifecycle event {'msg': 'downsampling leaves estimated 232161 word corpus (100.0%% of prior 232161)', 'datetime': '2021-11-18T20:21:52.768863', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'prepare_vocab'}
2021-11-18 20:21:52,971 : INFO : estimated required memory for 15908 words and 100 dimensions: 23922200 bytes
2021-11-18 20:21:52,973 : INFO : resetting layer weights
2021-11-18 20:21:52,995 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 2 workers on 15908 vocabulary and 100 features, using sg=1 hs=0 sample=0 negative=10 window=5 shrink_windows=True', 'datetime': '2021-11-18T20:21:52.995036', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[

In [ ]:
from gensim.models import KeyedVectors

from os import listdir
from os.path import isfile, join

names = [f for f in listdir(model_path) if isfile(join(model_path, f))]
print(names)
models = {}


for name in names:
    file_name = f"{model_path}/{name}"
    print(file_name)
    if not "npy" in name: ## !!! concat_x_y.model.syn1neg.npy cannot be loaded
      models[name] = Doc2Vec.load(file_name)

2021-11-18 20:23:07,478 : INFO : loading Doc2Vec object from ./models_doc2vec/autos_d2v_pv-dbow_3_10.model
2021-11-18 20:23:07,514 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_pv-dbow_3_10.model.dv.* with mmap=None
2021-11-18 20:23:07,516 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_pv-dbow_3_10.model.wv.* with mmap=None
2021-11-18 20:23:07,519 : INFO : setting ignored attribute cum_table to None


['autos_d2v_pv-dbow_3_10.model', 'autos_d2v_pv-dbow_3_4.model', 'autos_d2v_pv-dbow_3_3.model']
./models_doc2vec/autos_d2v_pv-dbow_3_10.model


2021-11-18 20:23:07,707 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_pv-dbow_3_10.model', 'datetime': '2021-11-18T20:23:07.706994', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}
2021-11-18 20:23:07,710 : INFO : loading Doc2Vec object from ./models_doc2vec/autos_d2v_pv-dbow_3_4.model
2021-11-18 20:23:07,762 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_pv-dbow_3_4.model.dv.* with mmap=None
2021-11-18 20:23:07,763 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_pv-dbow_3_4.model.wv.* with mmap=None
2021-11-18 20:23:07,765 : INFO : setting ignored attribute cum_table to None


./models_doc2vec/autos_d2v_pv-dbow_3_4.model


2021-11-18 20:23:07,991 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_pv-dbow_3_4.model', 'datetime': '2021-11-18T20:23:07.991182', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}
2021-11-18 20:23:07,992 : INFO : loading Doc2Vec object from ./models_doc2vec/autos_d2v_pv-dbow_3_3.model
2021-11-18 20:23:08,060 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_pv-dbow_3_3.model.dv.* with mmap=None
2021-11-18 20:23:08,063 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_pv-dbow_3_3.model.wv.* with mmap=None
2021-11-18 20:23:08,065 : INFO : setting ignored attribute cum_table to None


./models_doc2vec/autos_d2v_pv-dbow_3_3.model


2021-11-18 20:23:08,263 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_pv-dbow_3_3.model', 'datetime': '2021-11-18T20:23:08.263048', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}


In [ ]:
names = [f for f in listdir(data_path) if isfile(join(data_path, f))]
print(names)
data = {}

for name in names:
    file_name = f"{data_path}/{name}"
    print(file_name)
    with open(file_name, 'rb') as f:
        data[name] = pickle.load(f)

['autos_d2v_pv-dbow_3_3.train', 'autos_d2v_pv-dbow_3_10.train', 'autos_d2v_pv-dbow_3_4.train']
./traindata_doc2vec/autos_d2v_pv-dbow_3_3.train
./traindata_doc2vec/autos_d2v_pv-dbow_3_10.train
./traindata_doc2vec/autos_d2v_pv-dbow_3_4.train


#### Determine Similarity of Tunes

In [ ]:
#assert(len(model.dv.vectors == len(list(title_to_sectionid.keys()))))

In [ ]:
tunes_eval_list = [
  'These Foolish Things', 
]

In [ ]:
title_to_sectionid["These Foolish Things"]

[3866, 3867, 3868, 3869]

In [ ]:
df_sim = pd.DataFrame(columns=['title1', 
                               'model', 
                               'ref_section', 
                               'sim_section', 
                               'sim_value',
                               'sim_title'])

for tune in tunes_eval_list:
  print(tune)
  for section in title_to_sectionid[tune]:
      print(f"\n*** {sectionid_to_section[section]} ***")
      for model_name, model in models.items():
          print(f"{model_name}")
          sim = model.dv.most_similar(section)
          #print(f"'{sectionid_to_section[section]}' is most similar to:")
          for index, value in sim:
              print(f'\t{value:.3f}: {sectionid_to_section[index]}')

              # don't store results to the own tune
              #if index not in title_to_sectionid[tune]:
              if True:
                  df_sim.loc[len(df_sim)] = [tune, 
                                            model_name, 
                                            sectionid_to_section[section], 
                                            sectionid_to_section[index], 
                                            f"{value:.3f}",
                                             sectionid_to_title[index]]



These Foolish Things

*** These Foolish Things, section1 (A) ***
autos_d2v_pv-dbow_3_10.model
	0.915: These Foolish Things, section4 (A)
	0.889: These Foolish Things, section2 (A)
	0.865: Put On A Happy Face, section2 (B)
	0.798: Nearness Of You, The, section3 (B)
	0.778: Stairway To The Stars, section1 (A)
	0.773: It's A Lovely Day Today, section2 (B)
	0.767: On The Sunny Side Of The Street, section3 (B)
	0.762: Way You Look Tonight, The, section1 (A)
	0.761: Street Of Dreams, section2 (A)
	0.757: Emily, section2 (B)
autos_d2v_pv-dbow_3_4.model
	0.907: These Foolish Things, section4 (A)
	0.890: These Foolish Things, section2 (A)
	0.852: Put On A Happy Face, section2 (B)
	0.793: Nearness Of You, The, section3 (B)
	0.777: It's A Lovely Day Today, section2 (B)
	0.774: Stairway To The Stars, section1 (A)
	0.767: Way You Look Tonight, The, section1 (A)
	0.764: Street Of Dreams, section2 (A)
	0.758: Emily, section2 (B)
	0.754: On The Sunny Side Of The Street, section3 (B)
autos_d2v_pv-dbow_

In [ ]:
from google.colab import files

file = 'doc2vec_results.csv'
df_sim.to_csv(file)
files.download(file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_sim.sort_values(['sim_value'], ascending=False).head(50)

,title1,model,ref_section,sim_section,sim_value,sim_title
90,These Foolish Things,autos_d2v_pv-dbow_3_10.model,"These Foolish Things, section4 (A)","These Foolish Things, section2 (A)",0.938,These Foolish Things
30,These Foolish Things,autos_d2v_pv-dbow_3_10.model,"These Foolish Things, section2 (A)","These Foolish Things, section4 (A)",0.938,These Foolish Things
110,These Foolish Things,autos_d2v_pv-dbow_3_3.model,"These Foolish Things, section4 (A)","These Foolish Things, section2 (A)",0.936,These Foolish Things
50,These Foolish Things,autos_d2v_pv-dbow_3_3.model,"These Foolish Things, section2 (A)","These Foolish Things, section4 (A)",0.936,These Foolish Things
40,These Foolish Things,autos_d2v_pv-dbow_3_4.model,"These Foolish Things, section2 (A)","These Foolish Things, section4 (A)",0.931,These Foolish Things
100,These Foolish Things,autos_d2v_pv-dbow_3_4.model,"These Foolish Things, section4 (A)","These Foolish Things, section2 (A)",0.931,These Foolish Things
111,These Foolish Things,autos_d2v_pv-dbow_3_3.model,"These Foolish Things, section4 (A)","These Foolish Things, section1 (A)",0.921,These Foolish Things
20,These Foolish Things,autos_d2v_pv-dbow_3_3.model,"These Foolish Things, section1 (A)","These Foolish Things, section4 (A)",0.921,These Foolish Things
0,These Foolish Things,autos_d2v_pv-dbow_3_10.model,"These Foolish Things, section1 (A)","These Foolish Things, section4 (A)",0.915,These Foolish Things
91,These Foolish Things,autos_d2v_pv-dbow_3_10.model,"These Foolish Things, section4 (A)","These Foolish Things, section1 (A)",0.915,These Foolish Things


In [ ]:
df_sim.loc[:, 'model'].unique()

array(['autos_d2v_pv-dbow_3_10.model', 'autos_d2v_pv-dbow_3_4.model',
       'autos_d2v_pv-dbow_3_3.model'], dtype=object)

In [ ]:
dd = df_sim.query(f'model == "autos_d2v_pv-dm_4_4.model"').sort_values('sim_value', ascending=False).head(50)

In [ ]:
dd.value_counts('sim_title')

Series([], dtype: int64)

In [ ]:
for model_name in sorted(models):
  if "dbow_" in model_name:
    print(f"\n*** {model_name} ***")
    dd = df_sim.query(f'model == "{model_name}"').sort_values('sim_value', ascending=False).head(50)
    dd_count = dd.value_counts('sim_title')
    print(dd_count)


*** autos_d2v_pv-dbow_3_10.model ***
sim_title
These Foolish Things                 6
Emily                                4
Put On A Happy Face                  3
It's A Lovely Day Today              3
Way You Look Tonight, The            3
Rosetta                              2
Mountain Greenery                    2
Let's Fall In Love                   1
I Loves You Porgy                    1
I'm Getting Sentimental Over You     1
I've Told Eve'ry Little Star         1
If I Had You                         1
Imagination                          1
Isn't It A Pity                      1
Why Do I Love You                    1
More Than You Know                   1
Old Cape Cod                         1
On The Sunny Side Of The Street      1
Stairway To The Stars                1
Street Of Dreams                     1
They Can't Take That Away From Me    1
Too Young To Go Steady               1
Walkin' My Baby Back Home            1
Nearness Of You, The                 1
dtype: int64

**

Try out a model on multiple tunes

In [ ]:
best_model = 'autos_d2v_pv-dbow_3_4.model'
model = models[best_model]

In [ ]:
tunes_eval_list = [
  'Sweet Sue, Just You',
  'These Foolish Things', 
  'Blue Moon',
  'All Of Me',
  "All God's Chillun Got Rhythm",
  'I Got Rhythm',
  'Bye Bye Blackbird',
  'Old Fashioned Love',
  'Dinah',
  'Honeysuckle Rose',
  'Misty'
]

In [ ]:
df_sim = pd.DataFrame(columns=['title1', 
                               'model', 
                               'ref_section', 
                               'sim_section', 
                               'sim_value',
                               'sim_title'])

for tune in tunes_eval_list:
  print(tune)
  for section in title_to_sectionid[tune]:
      print(f"\n*** {sectionid_to_section[section]} ***")
      sim = model.dv.most_similar(section)
      #print(f"'{sectionid_to_section[section]}' is most similar to:")
      for index, value in sim:
          print(f'\t{value:.3f}: {sectionid_to_section[index]}')

          # don't store results to the own tune
          #if index not in title_to_sectionid[tune]:
          if True:
              df_sim.loc[len(df_sim)] = [tune, 
                                        model_name, 
                                        sectionid_to_section[section], 
                                        sectionid_to_section[index], 
                                        f"{value:.3f}",
                                          sectionid_to_title[index]]



Sweet Sue, Just You

*** Sweet Sue, Just You, section1 (A) ***
	0.998: Sweet Sue, Just You, section2 (A)
	0.998: Sweet Sue, Just You, section4 (A)
	0.908: Moten Swing, section4 (A)
	0.904: Honeysuckle Rose, section2 (A)
	0.901: Moten Swing, section1 (A)
	0.897: Seven Come Eleven, section4 (A)
	0.891: Seven Come Eleven, section1 (A)
	0.890: Seven Come Eleven, section2 (A)
	0.878: We See, section3 (B)
	0.871: Perdido, section2 (A)

*** Sweet Sue, Just You, section2 (A) ***
	0.998: Sweet Sue, Just You, section1 (A)
	0.998: Sweet Sue, Just You, section4 (A)
	0.909: Moten Swing, section4 (A)
	0.908: Honeysuckle Rose, section2 (A)
	0.903: Moten Swing, section1 (A)
	0.897: Seven Come Eleven, section4 (A)
	0.890: Seven Come Eleven, section1 (A)
	0.890: Seven Come Eleven, section2 (A)
	0.877: We See, section3 (B)
	0.871: Perdido, section2 (A)

*** Sweet Sue, Just You, section3 (B) ***
	0.847: Sweet Sue, section3 (B)
	0.814: Easy To Love, section2 (B)
	0.735: Save Your Love For Me, section2 (A)


# Good Matches

* Misty - Portrait Of Jennie
* Misty - September in the Rain
* Misty - I May Be Wrong


